In [3]:
import pandas as pd

In [4]:
import flask

In [5]:
import gunicorn

In [6]:
import requests

In [7]:
import sqlite3

In [8]:
conn = sqlite3.connect('data/chinook.db')

In [13]:
data = pd.read_sql_query("SELECT (c.firstname || ' ' || c.lastname) as FullName, \
i.BillingCountry as Country, i.InvoiceDate, g.Name as Genre, it.Quantity, round(i.Total,2) as Total \
FROM customers as c \
LEFT JOIN invoices as i ON i.customerid = c.customerid \
LEFT JOIN invoice_items as it ON it.invoiceid = i.invoiceid \
LEFT JOIN tracks as t ON t.trackid = it.trackid \
LEFT JOIN genres as g ON g.genreid = t.genreid", conn, parse_dates= 'InvoiceDate')
data['Month'] = data['InvoiceDate'].dt.month_name()
monthorder = ['January','February','March','April','May','June','July','August','September','October','November','December']
data['Month'] = pd.Categorical(data['Month'],
                                         categories=monthorder,
                                         ordered=True)
data.head()

,FullName,Country,InvoiceDate,Genre,Quantity,Total,Month
0,Luís Gonçalves,Brazil,2010-03-11,Sci Fi & Fantasy,1,3.98,March
1,Luís Gonçalves,Brazil,2010-03-11,Sci Fi & Fantasy,1,3.98,March
2,Luís Gonçalves,Brazil,2010-06-13,Rock,1,3.96,June
3,Luís Gonçalves,Brazil,2010-06-13,Rock,1,3.96,June
4,Luís Gonçalves,Brazil,2010-06-13,Rock,1,3.96,June


In [91]:
genre_country = pd.pivot_table(data=data,index='Month',columns=['Country','Genre'],values='Total', aggfunc=sum)
genre_country = genre_country.unstack(level=0).dropna()
genre_country

Country         Genre               Month    
Argentina       Alternative & Punk  March        27.72
                                    September     7.92
                                    November     44.55
                Easy Listening      March        27.72
                Jazz                January       3.96
                                                 ...  
United Kingdom  Rock                June         91.08
                                    July          1.98
                                    August       27.72
                                    October      83.16
                World               October      13.86
Length: 573, dtype: float64

In [45]:
sales = pd.pivot_table(data=data,
                                   index='FullName',
                                   columns='Genre',
                                  values='Total',
                                  aggfunc=sum)
sales = sales.reset_index().melt(id_vars = 'FullName')
sales = sales[sales['value'].notna()]
sales


,FullName,Genre,value
3,Bjørn Hansen,Alternative,63.44
16,Frank Ralston,Alternative,79.30
24,Isabelle Mercier,Alternative,67.44
46,Niklas Schröder,Alternative,0.99
60,Alexandre Rocha,Alternative & Punk,45.54
...,...,...,...
1387,João Fernandes,World,27.72
1399,Mark Philips,World,13.86
1401,Martha Silk,World,27.72
1409,Roberto Almeida,World,27.72
